In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# 各パス指定

In [19]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier2.keras'

# 分類数設定

In [7]:
NUM_CLASSES = 34
print(NUM_CLASSES)

34


# 学習データ読み込み

In [8]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# モデル構築

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [11]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 34)                374       
                                                                 
Total params: 1,444
Trainable params: 1,444
Non-trainable params: 0
_________________________________________________________________


In [9]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [12]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# モデル訓練

In [14]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    # callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
10/10 [==============================] - 1s 21ms/step - loss: 3.5575 - accuracy: 0.0223 - val_loss: 3.5226 - val_accuracy: 0.0335
Epoch 2/1000
10/10 [==============================] - 0s 5ms/step - loss: 3.5320 - accuracy: 0.0344 - val_loss: 3.5149 - val_accuracy: 0.0103
Epoch 3/1000
10/10 [==============================] - 0s 5ms/step - loss: 3.5217 - accuracy: 0.0541 - val_loss: 3.5111 - val_accuracy: 0.0696
Epoch 4/1000
10/10 [==============================] - 0s 5ms/step - loss: 3.5130 - accuracy: 0.0378 - val_loss: 3.5068 - val_accuracy: 0.0619
Epoch 5/1000
10/10 [==============================] - 0s 5ms/step - loss: 3.5123 - accuracy: 0.0447 - val_loss: 3.5029 - val_accuracy: 0.0593
Epoch 6/1000
10/10 [==============================] - 0s 4ms/step - loss: 3.5006 - accuracy: 0.0412 - val_loss: 3.4984 - val_accuracy: 0.0619
Epoch 7/1000
10/10 [==============================] - 0s 4ms/step - loss: 3.4965 - accuracy: 0.0412 - val_loss: 3.4924 - val_accuracy: 0.0696
Epoch

In [15]:
# モデル評価
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

4/4 [==============================] - 0s 1ms/step - loss: 1.1310 - accuracy: 0.7113


In [22]:
# 保存したモデルのロード
model = tf.keras.models.load_model(model_save_path)

In [25]:
# 推論テスト
predict_result = model.predict(np.array([X_test[5]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 19ms/step
[4.70245095e-06 6.85692867e-05 4.02165204e-03 6.75243151e-04
 3.16210717e-01 9.11484957e-02 8.04297626e-02 7.01153576e-02
 2.90944763e-02 4.49566953e-02 5.77019928e-05 2.35167488e-01
 2.61011068e-04 1.41126895e-03 8.84459761e-04 5.31339310e-02
 2.13115050e-06 3.85819794e-06 2.37465589e-04 2.74521206e-03
 3.25554574e-05 4.75445995e-05 7.43164628e-06 2.40668351e-06
 4.39511268e-06 8.26579526e-06 4.19389596e-03 3.71391725e-05
 1.11810177e-05 7.04263709e-03 7.65638566e-03 5.01257889e-02
 1.00213045e-04 9.99664917e-05]
4


# 混同行列

# Tensorflow-Lite用のモデルへ変換

In [21]:
# 推論専用のモデルとして保存
model.save(model_save_path, include_optimizer=False)

In [17]:
# モデルを変換(量子化)
# tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_quantized_model = converter.convert()

# open(tflite_save_path, 'wb').write(tflite_quantized_model)

# 推論テスト

In [18]:
# interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
# interpreter.allocate_tensors()

In [19]:
# # 入出力テンソルを取得
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

In [20]:
# interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [21]:
# %%time
# # 推論実施
# interpreter.invoke()
# tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [22]:
# print(np.squeeze(tflite_results))
# print(np.argmax(np.squeeze(tflite_results)))